# Block Encoding of a hermitian matrix A

Let's see how the BE of an matrix A is computed through the different methods explored in BE_general.ipynb

The Hermitian matrix that is going to be used as example for numerical results is the following:

$$ 
A =
\begin{pmatrix} 
0 & 1 & 0  & 0 \\ 
1 & -0.2 & 2 & 1 \\ 
0 & 2 & 0 & -6 \\ 
0 & 1 & -6  & 0 
\end{pmatrix} 
$$

In the end we will get:

$$
U_A = 
\begin{pmatrix}
A / \alpha & * \\
* & *
\end{pmatrix}
$$



---

(2.2 omitted)

In [16]:
import numpy as np
import pandas as pd

import pennylane as qml
from IPython.display import Image, display
import matplotlib.pyplot as plt
from pennylane.templates.state_preparations.mottonen import compute_theta, gray_code


from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, Statevector
from scipy.linalg import svd, sqrtm

A = np.array(
    [[0,   1, 0,  0],
     [1,-0.2, 2,  1],
     [0,   2, 0,  6],
     [0,   1, 6,  0]], dtype=np.complex128)   
     

## 1. LCU (Pennylane)

In [17]:
if not np.allclose(A, A.conj().T):
    U1 = None
    alpha1 = None
    block1 = None
else:
    LCU = qml.pauli_decompose(A)   
    LCU_coeffs, LCU_ops = LCU.terms()

    # Calculate how many target qubits n are going to be needed for this example
    n = int(np.log2(A.shape[0]))            # A is 2^n x 2^n

    # Had to add this absolute value because some coeffs were negative. 
    weights = np.abs(LCU_coeffs)
    alphas  = np.sqrt(weights) / np.linalg.norm(np.sqrt(weights))    # PREP amplitudes from absolute coefficients
    # Then rescue the sign in the unitaries
    def signed_op(op, c):  # Fold signs into the unitaries: U_i' = sign(c_i) * U_i 
        return op if c >= 0 else qml.s_prod(-1.0, op)  # -U is also unitary

    # Calculate how many ancilla qubits m are going to be needed for this example
    k = len(LCU_ops)
    m = int(np.ceil(np.log2(k))) # 2^m >= number of terms in LCU
    assert 2**m >= k, "Not enough ancilla qubits for the LCU terms"

    # Define wires (targets after ancillas) and remap 
    ancilla_wires = list(range(m))               # [0..m-1]
    target_wires  = list(range(m, m+n))          # [m .. m+n-1]
    mapping = {i: target_wires[i] for i in range(n)}
    unitaries = [qml.map_wires(signed_op(op,c), mapping) for c, op in zip(LCU_coeffs,LCU_ops)]

    # Pad to a power of two, both unitaries and amplitudes (with zeros and identities)
    if k < 2**m:
        pad = 2**m - k
        unitaries += [qml.Identity(wires=target_wires)] * pad
        alphas = np.concatenate([alphas, np.zeros(pad)])


    dev1 = qml.device("default.qubit", wires = m+n)
    @qml.qnode(dev1)                                
    def lcu_circuit():                              
        # PREP
        qml.StatePrep(alphas, ancilla_wires)             

        # SEL
        qml.Select(unitaries, control=ancilla_wires)         

        # PREP_dagger
        qml.adjoint(qml.StatePrep(alphas, ancilla_wires))
        return qml.state()                           

    U1 = qml.matrix(lcu_circuit)()
    print("Block-encoded A.1:\n",np.round(U1,4))



    # The block-encoding scale is alpha = sum |c_i|
    alpha1 = np.sum(weights)
    print("\nalpha (sum |c_i|) =", alpha1)
    # Extract |0...0> ancilla block -> should be A/alpha acting on targets
    dim_t = 2**n
    # With wires ordered [ancillas..., targets...], the top-left dim_t block corresponds to ancillas=|0...0>
    block1 = U1[:dim_t, :dim_t]
    print("\nApprox A/alpha block (real part):")
    print(np.round(block1, 6))
    print("\nalpha * block (should be ~ A):")
    print(np.round(alpha1 * block1, 6))

Block-encoded A.1:
 [[-0.000e+00+0.j  1.087e-01+0.j -0.000e+00+0.j ... -8.700e-03+0.j
  -2.920e-02+0.j -7.330e-02+0.j]
 [ 1.087e-01+0.j -2.170e-02+0.j  2.174e-01+0.j ...  3.540e-02+0.j
  -2.770e-02+0.j  7.180e-02+0.j]
 [-0.000e+00+0.j  2.174e-01+0.j  0.000e+00+0.j ... -2.770e-02+0.j
  -5.290e-02+0.j -8.700e-03+0.j]
 ...
 [-8.700e-03+0.j  3.540e-02+0.j -2.770e-02+0.j ...  5.294e-01+0.j
   2.820e-02+0.j  5.530e-02+0.j]
 [-2.920e-02+0.j -2.770e-02+0.j -5.290e-02+0.j ...  2.820e-02+0.j
   8.882e-01+0.j  2.000e-04+0.j]
 [-7.330e-02+0.j  7.180e-02+0.j -8.700e-03+0.j ...  5.530e-02+0.j
   2.000e-04+0.j  7.208e-01+0.j]]

alpha (sum |c_i|) = 9.2

Approx A/alpha block (real part):
[[-0.      +0.j  0.108696+0.j -0.      +0.j  0.      +0.j]
 [ 0.108696+0.j -0.021739+0.j  0.217391+0.j  0.108696+0.j]
 [-0.      +0.j  0.217391+0.j  0.      +0.j  0.652174+0.j]
 [ 0.      +0.j  0.108696+0.j  0.652174+0.j -0.      +0.j]]

alpha * block (should be ~ A):
[[-0. +0.j  1. +0.j -0. +0.j  0. +0.j]
 [ 1. +0.j -

## 2. Matrix Access Oracle (Pennylane)
### 2.1 Structured matrix

In [18]:
# Ensure A entries are in [-1, 1] for arccos; rescale if needed
A_max = np.max(np.abs(A))
if A_max > 1:
    print(f"Rescaling A by factor {A_max} so entries lie in [-1, 1].")
    A_scaled = A / A_max          
else:
    A_scaled = A



alphas = np.arccos(A_scaled).flatten() 
thetas = compute_theta(alphas)

ancilla_wires = ["ancilla"]

s = int(np.log2(A.shape[0]))                   # log2 of the number of rows/columns of A
wires_i = [f"i{index}" for index in range(s)]   # ['i0', 'i1'] for s=2
wires_j = [f"j{index}" for index in range(s)]   # ['j0', 'j1'] for s=2

# Then obtain the control wires for the C-NOT gates and a wire map that we later use to translate the control wires into the wire registers we prepared.
code = gray_code(int(2 * np.log2(len(A))))      # ['0000', '0001', '0011', '0010', '0110', '0111', '0101', '0100', '1100', '1101', '1111', '1110', '1010', '1011', '1001', '1000']
# Gray code as integers
def gray_code_int(n):
    return np.array([i ^ (i >> 1) for i in range(2**n)], dtype=int)
code = gray_code_int(int(2 * np.log2(len(A))))      # [ 0  1  3  2  6  7  5  4 12 13 15 14 10 11  9  8]
control_wires = np.log2(code ^ np.roll(code, -1)).astype(int) # Identify which bit flips (CNOT is applied on these wires)
wire_map = {control_index : wire for control_index, wire in enumerate(wires_j + wires_i)} # Map bit indices to actual wire labels

def UA(thetas, control_wires, ancilla):                                       # Apply the sequence of controlled rotations and C-NOT gates
    for theta, control_index in zip(thetas, control_wires):
        qml.RY(2 * theta, wires=ancilla)
        qml.CNOT(wires=[wire_map[control_index]] + ancilla)
def UB(wires_i, wires_j):                                                     # Swap the two registers
    for w_i, w_j in zip(wires_i, wires_j):
        qml.SWAP(wires=[w_i, w_j])
def HN(input_wires):                                                          # Apply Hadamard to all qubits in input_wires 
    for w in input_wires:
        qml.Hadamard(wires=w)
     
dev2 = qml.device('default.qubit', wires=ancilla_wires + wires_i + wires_j)    # We construct the circuit using these oracles and draw it.
@qml.qnode(dev2)                                                               # Creates a function that runs on the device 'dev'
def circuit():
    HN(wires_i)
    qml.Barrier()                                                             # To separate the sections in the circuit
    UA(thetas, control_wires, ancilla_wires)
    qml.Barrier()
    UB(wires_i, wires_j)
    qml.Barrier()
    HN(wires_i)
    return qml.probs(wires=ancilla_wires + wires_i)

wire_order = ancilla_wires + wires_i[::-1] + wires_j[::-1]
U2 = qml.matrix(circuit, wire_order=wire_order)().real
print("\nBlock-encoded A.2:\n",np.round(U2,4))
# The block-encoding scale is alpha = max singular value of A = ||A||max
alpha2 = len(A)
print("\nalpha (matrix dim.) =", alpha2)
# Extract |0...0> ancilla block -> should be A/alpha acting on targets
dim_t = 2**s
# With wires ordered [ancillas..., targets...], the top-left dim_t block corresponds to ancillas=|0...0>
block2 = U2[:dim_t, :dim_t]
print("\nApprox A/alpha block (real part):")
print(np.round(block2, 6))


recovered = alpha2 * block2
if A_max > 1:
    recovered = A_max * recovered
print("\nalpha * block * scaling (should be ~ A):")
print(np.round(recovered, 6))


Rescaling A by factor 6.0 so entries lie in [-1, 1].

Block-encoded A.2:
 [[-0.      0.0417 -0.     ... -0.2465 -0.25   -0.25  ]
 [ 0.0417 -0.0083  0.0833 ...  0.2499  0.2357  0.2465]
 [-0.      0.0833  0.     ...  0.2357  0.25   -0.    ]
 ...
 [ 0.2465 -0.2499 -0.2357 ... -0.0083  0.0833 -0.0417]
 [ 0.25   -0.2357 -0.25   ...  0.0833 -0.     -0.25  ]
 [ 0.25   -0.2465 -0.     ... -0.0417 -0.25    0.    ]]

alpha (matrix dim.) = 4

Approx A/alpha block (real part):
[[-0.        0.041667 -0.        0.      ]
 [ 0.041667 -0.008333  0.083333  0.041667]
 [-0.        0.083333  0.        0.25    ]
 [-0.        0.041667  0.25      0.      ]]

alpha * block * scaling (should be ~ A):
[[-0.   1.  -0.   0. ]
 [ 1.  -0.2  2.   1. ]
 [-0.   2.   0.   6. ]
 [-0.   1.   6.   0. ]]


### 2.2 Sparse matrix (omitted because it does not apply here)

## 3. Defined function (Pennylane)

In [19]:
op = qml.BlockEncode(A, wires=range(3)) 
U3 = qml.matrix(op)
alpha3 = op.hyperparameters["norm"]

print("Block-encoded A.3:\n",np.round(U3,4))
print("\nalpha (spect norm) =", alpha3)
# Extract |0...0> ancilla block -> should be A/alpha acting on targets
dim_t = A.shape[0]
# With wires ordered [ancillas..., targets...], the top-left dim_t block corresponds to ancillas=|0...0>
block3 = U3[:dim_t, :dim_t]
print("\nApprox A/alpha block (real part):")
print(np.round(block3, 6))
print("\nalpha * block (should be ~ A):")
print(np.round(alpha3 * block3, 6))

Block-encoded A.3:
 [[ 0.000e+00+0.j  1.930e-02+0.j  0.000e+00+0.j  0.000e+00+0.j
   9.998e-01+0.j  0.000e+00+0.j -4.000e-04+0.j -2.000e-04+0.j]
 [ 1.930e-02+0.j -3.900e-03+0.j  3.860e-02+0.j  1.930e-02+0.j
   0.000e+00+0.j  9.989e-01+0.j -1.000e-03+0.j -2.200e-03+0.j]
 [ 0.000e+00+0.j  3.860e-02+0.j  0.000e+00+0.j  1.158e-01+0.j
  -4.000e-04+0.j -1.000e-03+0.j  9.925e-01+0.j -4.000e-04+0.j]
 [ 0.000e+00+0.j  1.930e-02+0.j  1.158e-01+0.j  0.000e+00+0.j
  -2.000e-04+0.j -2.200e-03+0.j -4.000e-04+0.j  9.931e-01+0.j]
 [ 9.998e-01+0.j  0.000e+00+0.j -4.000e-04+0.j -2.000e-04+0.j
  -0.000e+00+0.j -1.930e-02+0.j -0.000e+00+0.j -0.000e+00+0.j]
 [ 0.000e+00+0.j  9.989e-01+0.j -1.000e-03+0.j -2.200e-03+0.j
  -1.930e-02+0.j  3.900e-03+0.j -3.860e-02+0.j -1.930e-02+0.j]
 [-4.000e-04+0.j -1.000e-03+0.j  9.925e-01+0.j -4.000e-04+0.j
  -0.000e+00+0.j -3.860e-02+0.j -0.000e+00+0.j -1.158e-01+0.j]
 [-2.000e-04+0.j -2.200e-03+0.j -4.000e-04+0.j  9.931e-01+0.j
  -0.000e+00+0.j -1.930e-02+0.j -1.158e-01+

## 4. Definition of BE (Qiskit)

In [20]:
# Compute normalization factor alpha (largest singular value)
_, s_vals, _ = svd(A)
alpha4 = max(s_vals) # Largest singular value is spectral norm
A_norm = A / alpha4

# Build U(A) in block form
I2 = np.eye(A.shape[0])
AA_dag = A_norm @ A_norm.conj().T
A_dagA = A_norm.conj().T @ A_norm

block_upper_right = sqrtm(I2 - AA_dag)
block_lower_left = sqrtm(I2 - A_dagA)

top = np.hstack([A_norm, block_upper_right])
bottom = np.hstack([block_lower_left, -A_norm.conj().T])
U4 = np.vstack([top, bottom])

print("Block-encoded A.4:\n",np.round(U4,4))
print("\nalpha (spect norm) =", alpha4)
# Extract |0...0> ancilla block -> should be A/alpha acting on targets
dim_t = A.shape[0]
# With wires ordered [ancillas..., targets...], the top-left dim_t block corresponds to ancillas=|0...0>
block4 = U4[:dim_t, :dim_t]
print("\nApprox A/alpha block (real part):")
print(np.round(block4, 6))
print("\nalpha * block (should be ~ A):")
print(np.round(alpha4 * block4, 6))


# # EXTRA: Make U exactly unitary (it is only approximately so due to numerical errors)
# def nearest_unitary(M):
#     U, _, Vh = svd(M)
#     return U @ Vh
# U5_approx = nearest_unitary(U5)
# print("\nBlock-encoded A.5 (with approximate nearest unitary):\n",np.real(np.round(U5_approx,4)))

Block-encoded A.4:
 [[ 0.    +0.j  0.1499+0.j  0.    +0.j  0.    +0.j  0.9876+0.j -0.004 +0.j
  -0.0407+0.j -0.0237+0.j]
 [ 0.1499+0.j -0.03  +0.j  0.2997+0.j  0.1499+0.j -0.004 +0.j  0.8834+0.j
  -0.1499+0.j -0.248 +0.j]
 [ 0.    +0.j  0.2997+0.j  0.    +0.j  0.8991+0.j -0.0407+0.j -0.1499+0.j
   0.2261+0.j -0.1629+0.j]
 [ 0.    +0.j  0.1499+0.j  0.8991+0.j  0.    +0.j -0.0237+0.j -0.248 +0.j
  -0.1629+0.j  0.2838+0.j]
 [ 0.9876+0.j -0.004 +0.j -0.0407+0.j -0.0237+0.j -0.    +0.j -0.1499+0.j
  -0.    +0.j -0.    +0.j]
 [-0.004 +0.j  0.8834+0.j -0.1499+0.j -0.248 +0.j -0.1499+0.j  0.03  +0.j
  -0.2997+0.j -0.1499+0.j]
 [-0.0407+0.j -0.1499+0.j  0.2261+0.j -0.1629+0.j -0.    +0.j -0.2997+0.j
  -0.    +0.j -0.8991+0.j]
 [-0.0237+0.j -0.248 +0.j -0.1629+0.j  0.2838+0.j -0.    +0.j -0.1499+0.j
  -0.8991+0.j -0.    +0.j]]

alpha (spect norm) = 6.673254632700024

Approx A/alpha block (real part):
[[ 0.      +0.j  0.149852+0.j  0.      +0.j  0.      +0.j]
 [ 0.149852+0.j -0.02997 +0.j  0.2997

In [21]:
# In order to work even with non-Hermitian A, we set U1, alpha1, block1 to None at the beginning if A is not Hermitian.
def shape_or_none(U):
    return None if U is None else U.shape

def is_hermitian_or_none(U):
    return None if U is None else np.allclose(U, U.conj().T)

def is_unitary_or_none(U):
    return None if U is None else np.allclose(U @ U.conj().T, np.eye(U.shape[0]))

def correctness_or_none(A, alpha, block):
    return None if (alpha is None or block is None) else np.allclose(A, alpha * block)

def error_norm_or_none(A, alpha, block):
    return None if (alpha is None or block is None) else np.linalg.norm(A - alpha * block)



In [22]:
# CHECKS

data = {
    "Method": ["LCU", "Demo", "BlockEncode op", "SVD spectral norm"],
    "Needs Hermitian A?": [True, False, False, False],
    "Alpha": [alpha1, alpha2, alpha3, alpha4],
    "Correctness (allclose)": [
        correctness_or_none(A, alpha1, block1),
        correctness_or_none(A, alpha2 * A_max, block2),
        correctness_or_none(A, alpha3, block3),
        correctness_or_none(A, alpha4, block4),
    ],
    "Error norm": [
        error_norm_or_none(A, alpha1, block1),
        error_norm_or_none(A, alpha2 * A_max, block2),
        error_norm_or_none(A, alpha3, block3),
        error_norm_or_none(A, alpha4, block4),
    ],
    "U shape": [
        shape_or_none(U1), shape_or_none(U2), shape_or_none(U3), shape_or_none(U4)
    ],
    "U Hermitian": [
        is_hermitian_or_none(U1),
        is_hermitian_or_none(U2),
        is_hermitian_or_none(U3),
        is_hermitian_or_none(U4),
    ],
    "U Unitary": [
        is_unitary_or_none(U1),
        is_unitary_or_none(U2),
        is_unitary_or_none(U3),
        is_unitary_or_none(U4),
    ],
    
}

df = pd.DataFrame(data, index=["U1", "U2", "U3", "U4"])
print(df)


               Method  Needs Hermitian A?      Alpha  Correctness (allclose)  \
U1                LCU                True   9.200000                    True   
U2               Demo               False   4.000000                    True   
U3     BlockEncode op               False  51.800000                    True   
U4  SVD spectral norm               False   6.673255                    True   

      Error norm   U shape  U Hermitian  U Unitary  
U1  2.332350e-15  (64, 64)         True       True  
U2  5.911158e-15  (32, 32)        False       True  
U3  1.313927e-15    (8, 8)         True       True  
U4  0.000000e+00    (8, 8)         True       True  
